## Kratak uvod u RAG koristeći LangChain i HuggingFace

**Retrieval-augmented generation**, skraćeno RAG, je tehnika koja poboljšava pouzdanost generativnih modela sa mogućnostima pronalaženja (retrieval).

RAG koristimo da dopunimo velike jezičke modele, skraćeno VJM tj. LLM - Large Language Model, tako što im na taj način omogućavamo pristup relevantnim informacijama iz spoljnih izvora, poboljšavajući time sposobnost VJM modela da generiše odgovore koji su relevantni i tačni za zadane upite.

Pogledajte ilustraciju:

![picture](https://drive.google.com/uc?id=1qRU-B8TjYDz8ANRB7Y1lPwIRdehijurL)

**Izvor za ilustraciju:** [DataKolektiv](https://datakolektiv.com/)



1. **Embeddings**  
  *Embeddings* predstavlja vektorsku reprezentaciju, u našem slučaju, teksta.
2. **Prompt + Query**  
  *Prompt* predstavlja početni upit koju korisnik postavlja LLM modelu, a *query* je konkretno pitanje ili zahtjev koji se postavlja.
3. **Query Embedding**  
  *Query Embedding* je vektorska reprezentacija upita. Ovaj vektor se koristi za pretraživanje relevantnih dokumenata u vektorskoj reprezentaciji.
4. **Relevant Context Documents**  
  *Relevant Context Documents* su dokumenti ili informacije iz baze koje su relevantne za postavljeni upit.
5. **Enhanced Prompt**  
  *Enhanced Prompt* je verzija osnovnog prompta koja uključuje informacije iz relevantnih dokumenata, odnosno sadrži kontekst koji će LLM-u pomoći prilikom generisanja odgovora. Ovaj odgovor je rezultat RAG sistema koji kombinuje informacije iz početnog prompta, korisničkog upita i odgovarajućeg konteksta.

## Zadatak

Vaš zadatak je da kreirate RAG koristeći HuggingFace (za javno dostupne modele) i LangChain.

**Koraci:**

0. Instalacija potrebnih paketa
1. Učitavanje i priprema podataka (dokumenata)
2. Kreiranje vektorske baze i retrivera  
3. Učitavanje LLM modela
4. Definisanje pipeline-a


**Napomene:**


*   Koristite dokumentaciju:
  * [HuggingFace Transformers](https://huggingface.co/docs/transformers/en/index)
  * [HuggingFace Hub](https://huggingface.co/docs/hub/index)
  * [LangChain](https://python.langchain.com/docs/get_started/introduction)
*   Možete koristiti ChatGPT i Copilot (ex. Bing Chat) 🤖

#### Instalacija potrebnih paketa

In [ ]:
!pip install -q torch transformers accelerate bitsandbytes sentence-transformers faiss-gpu
!pip install -q langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.5/149.5 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.4/246.4 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 20.4 MB/s eta 0:00:00


In [ ]:
!pip install -q arxiv
!pip install -q pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 37.1 MB/s eta 0:00:00


#### Učitavanje i priprema podataka

Koristite *LangChain* za učitavanje dokumenata ili nekog drugog tekstualnog sadržaja nad kojim želite izvršavati upite - [link](https://python.langchain.com/docs/integrations/document_loaders/).

Kada odaberete podatke koje ćete koristiti, potrebno je da ih pripremite na odgovarajući način za dalju obradu i čuvanje u vektorskoj bazi.


In [ ]:
from langchain.document_loaders import ArxivLoader

document_id = "1706.03762v1" # inicijalna verzija rada Attention Is All You Need
loader = ArxivLoader(query=document_id, load_max_docs=2)
docs = loader.load()

In [ ]:
# Published - datum kada je dokument objavljen ili posljednji put izmijenjen
docs[0].metadata

{'Published': '2017-06-12',
 'Title': 'Attention Is All You Need',
 'Authors': 'Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin',
 'Summary': 'The dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks in an encoder-decoder configuration. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer, based\nsolely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks show these models to be\nsuperior in quality while being more parallelizable and requiring significantly\nless time to train. Our model achieves 28.4 BLEU on the WMT 2014\nEnglish-to-German translation task, improving over the existing best results,\nincluding ensembles by over 2 BLEU. On the WMT 2014 English-to-French\ntranslation task, 

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

doc_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=20)

chunked_docs = doc_splitter.split_documents(docs)

In [ ]:
len(chunked_docs)

79

#### Kreiranje vektorske baze i *retriever*-a

Izaberite vekrorsku bazu koju ćete koristiti za čuvanje vektora (embedding-a) dokumenata, pregled podržanih baza možete pronaći na ovom [linku](https://python.langchain.com/docs/integrations/vectorstores/).

Vektore trebate kreirati koristeći otvorene embedding modele, potražite neki manji embedding model na HuggingFace Hub-u, referišite se na [Massive Text Embedding Benchmark (MTEB) Leaderboard](https://huggingface.co/spaces/mteb/leaderboard).

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy

# ovdje smo koristili osnovni bge (BAAI general embedding) model
# dimenzija vektora ovog modela je 768
embeddings_model = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")
VECTOR_DB = FAISS.from_documents(chunked_docs, embeddings_model, distance_strategy=DistanceStrategy.COSINE)

In [ ]:
retriever = VECTOR_DB.as_retriever(search_type="similarity", search_kwargs={"k": 3})

#### Učitavanje LLM-a

Potražite neki manji LLM model na HuggingFace Hub-u, referišite se na [Open LLM Leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard). Cilj je da iskoristite model koji je optimizovan za brže izvršavanje.

Prilikom instanciranja modela, možete koristiti konfiguracije koje optimizuju njegovo izvršavanje kao što je kvantizacija (quantization).


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# ucitavamo optimizovanu tj. kvantizovanu verziju LLM modela, radi brzeg izvrsavanja
# LLM model koji koristimo je zasnovan na Mistral modelu
model_name = "HuggingFaceH4/zephyr-7b-beta"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

llm_model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

####  Definisanje pipeline-a

Koristite HuggingFace *transformers* biblioteku za konstruisanje lanca ili pipeline-a za generisanje teksta, ovaj korak uključuje postavljanje parametara kao što su prethodno definisani LLM model, tokenizator, temperatura i drugi.

Potrebno je da definišete šablon za unos koristeći *PromptTemplate*, na ovaj način modelu dajete potrebne informacije o kontekstu i pitanju koje se postavlja.

Na kraju je potrebno da povežete sve komponente lanca.

In [ ]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

text_generation_pipeline = pipeline(
    model=llm_model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=400,
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# prompts
"""
#1
Using the information contained in the context, give a comprehensive answer to
the question. Respond only to the question asked, response should be concise and
 relevant to the question. Context is:

#2
Your task is to answer to the given question based on your knowledge. Use the following context:

"""
prompt_template = """
<|system|>
Using the information contained in the context, give a comprehensive answer to
the question. Respond only to the question asked, response should be concise and
 relevant to the question. Context is:
{context}

</s>
<|user|>
{question}
</s>
<|assistant|>
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# povezivanje komponenata lanca
llm_chain = prompt | llm | output_parser

In [ ]:
from langchain_core.runnables import RunnablePassthrough

retriever = VECTOR_DB.as_retriever()

components = {
    "context": retriever,
    "question": RunnablePassthrough()
}

rag_chain = components | llm_chain

def invoke_rag_chain(question):
  response = rag_chain.invoke(question)

  return response

#### Testiranje

In [ ]:
question = "What is the name of proposed architecture?"

In [ ]:
invoke_rag_chain(question)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'The name of the proposed architecture is the Transformer, as described in the paper "Attention Is All You Need" by Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, and Illia Polosukhin (published on June 12, 2017). The Transformer is a simple network architecture based solely on attention mechanisms, which dispenses with recurrence and convolutions entirely and has shown superior performance in machine translation tasks compared to existing models.'

In [ ]:
question = "What is the main idea of attention?"

In [ ]:
invoke_rag_chain(question)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'In the context provided, the main idea of attention is a mechanism used in sequence transduction models, specifically in machine translation tasks, which allows the model to focus on specific parts of a sequence during processing. This mechanism, called self-attention, relates different positions of a single sequence to compute a representation of the sequence. It helps to improve the quality of the model\'s output while making it more parallelizable and requiring less time to train compared to traditional sequence transduction models based on complex recurrent or convolutional neural networks. The authors of the paper "Attention Is All You Need" propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. The paper presents experimental results showing that their model outperforms existing best results in machine translation tasks, including ensembles, by significant margins. Overall, attention

In [ ]:
question = "How did the writers evaluate the significance of certain architectural elements?"

In [ ]:
invoke_rag_chain(question)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'The writers evaluated the significance of certain architectural elements through experiments and analysis. They varied the number of attention heads and the attention key and value dimensions, as well as the attention key size dk, and observed the impact on model quality. They also noted that some attention heads seemed to be involved in anaphora resolution, as evidenced by sharp attentions for specific words. Additionally, they found that bigger models were better and dropout was helpful in avoiding overfitting. These observations led them to suggest that determining compatibility may require a more sophisticated compatibility function than dot product. Overall, their experiments and analyses provided insights into the importance and effectiveness of various architectural elements in their proposed network architecture, the Transformer.'